In [1]:
!pip install accelerate --quiet

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from tqdm import tqdm

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
data = json.load(open("/kaggle/input/visionpulse-inference/BLIP_VizWiz_Val.json"))

In [5]:
for i in tqdm(range(len(data))):
    Question = data[i]['question']
    Reference_answers = ', '.join([x['answer'] for x in data[i]['answers']])
    Candidate_answer = data[i]['prediction']
    PROMPT= f"Task description: \
           You are given a question, a set of gold-standard reference answers written by experts, and a candidate answer. Please rate the accuracy of the candidate answer for the question considering the reference answers. Use a scale of 1-3, with 1 indicating an incorrect or irrelevant answer, 2 indicating an ambiguous or incomplete answer, and 3 indicating a correct answer. Only give the rating. \
           Demonstrations: \
    Question: What is the color of the car? \
    Reference answers: red, red, red, red, scarlet \
    Candidate answer: pink \
    Output: The candidate answer is incorrect because the car is red and not pink. So rating=1 \
    Question: What is the animal on the left? \
    Reference answers: elephant, giraffe, giraffe, giraffe, giraffe \
    Candidate answer: giraffe \
    Output: The candidate answer is correct because most of the reference answers (4 out of 5) indicate the \
        animal on the left is a giraffe. So rating=3 \
    Question: Whats the weather like? \
    Reference answers: bright, bright and sunny, clear, sunny, sunny, sunny \
    Candidate answer: cloudy \
    Output: The candidate answer is incorrect because the weather is bright and sunny, not cloudy. So \
    rating=1 \
    Question: What are the people in the picture doing? \
    Reference answers: sitting, sitting, sitting, sitting \
    Candidate answer: they are resting \
    Output: The candidate answer is ambiguous because, while it is common that people who are sitting are \
        resting, it is not always the case. So rating=2 \
    Question: What color are the base tiles? \
    Reference answers: beige, beige, beige, brown, brown, tan, tan, tan, tan, ten \
    Candidate answer: brown \
    Output: The candidate answer is correct because the reference answers include brown and other similar \
        colors such as tan or beige. So rating=3 \
    Question: How many people are in the picture? \
    Reference answers: four, three, three, three, two, two \
    Candidate answer: a few \
    Output: The candidate answer is incomplete because a few is less specific than the numerical reference \
        answers. So rating=2 \
    Question: What type of fruit is in the picture? \
    Reference answers: apple \
    Candidate answer: fruit \
    Output: The candidate answer is incorrect because it does not specify the type of fruit. So rating=1 \
    Question: What type of sculpture is this? \
    Reference answers: Horse statue. \
    Candidate answer: horse \
    Output: The candidate answer is correct because horse is equivalent to horse statue in this context. \
    So rating=3 \
    Test example:\
    Question:{Question}\
    Reference answers: {Reference_answers}\
    Candidate answer:{Candidate_answer}\
    Output:"
    input_ids = tokenizer(PROMPT, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids)
    data[i]['LAVE'] = tokenizer.decode(outputs[0], skip_special_tokens=True)

  0%|          | 0/4319 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 4319/4319 [54:38<00:00,  1.32it/s]


In [7]:
json.dump(data, open(f'BLIP_VizWiz_Val_LAVE.json', 'w'), indent=4)

-----------------------------------------------------------------------------------

In [1]:
import json
data = json.load(open("BLIP_VizWiz_Val_LAVE.json"))
LAVE = [x['LAVE'] for x in data]

count, lave_score = {'1': 0, '2': 0, '3': 0}, {}
for i in LAVE:
    count[i] += 1
lave_score['incorrect'] = count['1']/len(LAVE)
lave_score['ambiguous'] = count['2']/len(LAVE)
lave_score['correct'] = count['3']/len(LAVE)

print(lave_score)

{'incorrect': 0.769391062746006, 'ambiguous': 0.0009261403102570039, 'correct': 0.22968279694373697}
